<a href="https://colab.research.google.com/github/monjar/AI-Assignments/blob/master/ClassicNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#imports
from math import exp
import numpy as np
from random import seed, random
import matplotlib.pyplot as plt

In [54]:
class NeuralNetwork:

  def __init__(self, n_inputs, n_hidden, n_outputs):
    self.network = []
    hidden_layer = [{'weights' : [random() for i in range (n_inputs + 1)]} for i in range(n_hidden)]
    output_layer = [{'weights' : [random() for i in range (n_hidden + 1)]} for i in range(n_outputs)]
    self.network.append(hidden_layer)
    self.network.append(output_layer)
    self.n_outputs = n_outputs

  def activate_function(self, weights, inputs):
    active_value = weights[-1] # Adding the bias
    for i in range(len(weights) - 1):
      active_value += weights[i] * inputs[i]
    return active_value

  def transfer(self, active_value):
    return 1.0 / (1.0 + exp(-active_value))

  def d_transfer(self, output):
    return output * (1.0 - output)

  def predict(self, inputs):
    prev_layer_output = inputs
    for layer in self.network:
      current_layer_output = []
      for neuron in layer:
        neuron['output'] = self.transfer(self.activate_function(neuron['weights'], prev_layer_output))
        current_layer_output.append(neuron['output'])
      prev_layer_output = current_layer_output
    return prev_layer_output

  def backward_prop_error(self, expected):
    for i in reversed(range(len(self.network))):
      layer = self.network[i]
      errors = []
      if i != len(self.network) - 1:
        for j in range(len(layer)):
          error = 0.0
          for neuron in self.network[i + 1]:
            error += (neuron['weights'][j] * neuron['delta'])
          errors.append(error)
      else:
        for j in range (len(layer)):
          neuron = layer[j]
          errors.append(expected[j] - neuron['output'])
      for j in range(len(layer)):
        neuron = layer[j]
        neuron['delta'] = errors[j] * self.d_transfer(neuron['output'])

  def update_weights(self, row, l_rate):
    for i in range(len(self.network)):
      inputs = row[:-1]
      if i !=0:
        inputs = [neuron['output'] for neuron in self.network[i-1]]

      for neuron in self.network[i]:
        for j in range(len(inputs)):
          neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
        neuron['weights'][-1] += l_rate * neuron['delta']


  def fit(self, train_data, learning_rate, n_epoch):
    for epoch in range(n_epoch):
      sum_error = 0
      for row in train_data:
        outputs = self.predict(row)
        expected = [0 for i in range(self.n_outputs)]
        expected[row[-1]] = 1
        sum_error =+ sum([(expected[i] - outputs[i]) ** 2 for i in range (len(expected))])
        self.backward_prop_error(expected)
        self.update_weights(row, learning_rate)
      if (epoch + 1) % 100 == 0:
        print(f"Epoch #{epoch + 1}/{n_epoch} ---- error: {sum_error}")

In [59]:
seed(1)
dataset = [[0.959,  0.085,  0.058,      1,    0.0,    0.0,  1],
           [0.981,  0.034,   0.04,  0.892,  0.093,  0.021,  0],
           [0.907,  0.034,  0.028,  0.895,  0.055,  0.055,  0], 
           [0.994,  0.039,  0.026,  0.881,  0.044,  0.082,  0],
           [0.999,  0.074,  0.068,   0.88,  0.043,  0.048,  0],
           [0.972,  0.019,  0.074,  0.949,  0.029,   0.08,  1], 
           [0.952,  0.024,  0.083,  0.893,  0.053,  0.015,  0],
           [0.903,  0.071,  0.078,  0.922,  0.035,  0.035,  1],
           [0.976,  0.092,  0.014,  0.918,  0.057,  0.051,  1],
           [0.883,   0.02,  0.001,  0.931,  0.018,  0.079,  1],
           [0.959,  0.055,  0.047,  0.914,  0.048,  0.018,  1],
           [0.94,   0.023,  0.008,  0.996,  0.067,  0.065,  1],
           [0.88,   0.066,  0.009,  0.895,  0.051,  0.077,  0],
           [0.911,  0.005,  0.002,  0.995,  0.025,  0.004,  1],
           [0.922,  0.057,  0.055,  0.881,  0.076,  0.026,  0],
           [0.904,  0.063,  0.097,  0.921,  0.098,   0.09,  0]]
x = np.array(dataset)[:,:-1]
y = np.array(dataset)[:,-1]

In [60]:
classic_nn = NeuralNetwork(len(x[0]), 10, len(set(y)))

In [61]:
classic_nn.fit(dataset, 0.5, 1000)

Epoch #100/1000 ---- error: 0.5890243534651546
Epoch #200/1000 ---- error: 0.5409852501334749
Epoch #300/1000 ---- error: 0.5185564948479626
Epoch #400/1000 ---- error: 0.4487875637607601
Epoch #500/1000 ---- error: 0.28665437433508806
Epoch #600/1000 ---- error: 0.154026874819074
Epoch #700/1000 ---- error: 0.09428873396171789
Epoch #800/1000 ---- error: 0.07243549866643086
Epoch #900/1000 ---- error: 0.06015282645625704
Epoch #1000/1000 ---- error: 0.040752976861115314


In [62]:
classic_nn.predict([0.976,  0.092,  0.014,  0.918,  0.057,  0.051])

[0.6267269343593079, 0.3750520709211139]